In [25]:
# Solve a generalized assignment problem between the different tracklets 
# setting similarity weights on the assignments such that we maximize the
# total assignment weigthed objective and match the most likely tracklets
from gurobipy import *
from numpy import *
import os
import glob

In [27]:
directory = os.path.join("c:\\","path")

for root,dirs,files in os.walk(directory):
    for file in files:
        if file.endswith(".csv"):
            f=open(file, 'r')
           #  perform calculation
            f.close()

In [79]:
import fnmatch
import os

matches = []
for root, dirnames, filenames in os.walk(cwd):
    for filename in fnmatch.filter(filenames, 'Pij_5.csv'):
        file_path = os.path.join(root, filename)
        P = loadtxt(open(root + '/' + filename, "rb"), delimiter=",")
        if root == '/home/david/Dropbox/Ubuntu/Convex_Optimization/Project/Experiments/3_obj/csv_3_11':
            print root
            print P
            numT = shape(P)[0]
            numC = shape(P)[1]

            T=range(numT)
            C=range(numC)

            # Initialize the model
            m=Model("Assignment")

            # Define the binary variables Xs
            X = []
            for t in T:
                X.append([])
                for c in C:
                    X[t].append(m.addVar(vtype=GRB.BINARY,name="X%d%d"% (t, c)))

            # Set the optimization direction
            m.update()
            m.modelSense = GRB.MAXIMIZE


            # Define the assignment constraints
            constraintT = []
            constraintC = []

            # One variable assigned per column
            for t in range(numT):
                constraintT.append(m.addConstr(quicksum(X[t][c] for c in range(numC))
                                               <= 1 ,'constraintT%d' % t))

            # One variable assigned per row
            for c in range(numC):
                constraintT.append(m.addConstr(quicksum(X[t][c] for t in range(numT))
                                               <= 1 ,'constraintC%d' % t))


            # Define objective
            m.setObjective(quicksum(quicksum([X[t][c]*P[t][c] 
                                    for c in range(numC)]) 
                                    for t in range(numT)))

            m.update()

            # Solve the optimization problem
            m.optimize()

            # Print runtime and solution

            print 'runtime is',m.Runtime, 's'

            Xsol = zeros((numT,numC))
            # Print solution
            if m.status == GRB.Status.OPTIMAL:
                for t in range(numT):
                    for c in range(numC):
                        if X[t][c].x > 0.5:
                            print('%g -> %g' % (t, c))
                            Xsol[t][c] = 1
            print Xsol
            name_sol = root + "/Assignment.csv"
            savetxt(name_sol, Xsol, delimiter=",")

/home/david/Dropbox/Ubuntu/Convex_Optimization/Project/Experiments/3_obj/csv_3_11
[[ -1.00000000e+03  -1.00000000e+03  -1.00000000e+03   5.20000000e-01
   -3.02220000e-01  -5.38440000e-01  -1.07170000e+00]
 [ -1.00000000e+03  -1.00000000e+03  -1.00000000e+03   2.02000000e+00
   -1.00000000e+03  -4.95150000e-01  -8.92700000e-01]
 [ -1.00000000e+03  -1.00000000e+03  -1.00000000e+03  -1.00000000e+03
   -1.00000000e+03  -8.71030000e-01  -1.10730000e+00]
 [ -1.00000000e+03  -1.00000000e+03  -1.00000000e+03  -1.00000000e+03
   -1.00000000e+03  -4.11820000e-01  -7.85560000e-01]]
Optimize a model with 11 rows, 28 columns and 56 nonzeros
Variable types: 0 continuous, 28 integer (28 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e-01, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 0.52
Presolve removed 11 rows and 28 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored

Optimize a model with 10 rows, 25 columns and 50 nonzeros
Variable types: 0 continuous, 25 integer (25 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e-01, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective -0

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: -0 

Optimal solution found (tolerance 1.00e-04)
Best objective -0.000000000000e+00, best bound -0.000000000000e+00, gap 0.0000%


runtime is 0.0195119380951 s
[[ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]]
/home/david/Dropbox/Ubuntu/Convex_Optimization/Project/Experiments/3_obj/csv_3_22
